In [1]:
from joblib import Parallel, delayed
import statsmodels.stats.power as smp
from scipy.stats import ttest_1samp
import pandas as pd
import numpy as np

from util.preprocessing import calc_overest_means
from util.simulation import DataSimulator

In [2]:
alpha_main = 0.01
alpha_inter = 0.05
p_aware = 0.05
n_simulations = 10000

In [3]:
# pick sample size to achieve target power for main effect
power_analysis = smp.TTestPower()
sample_size = power_analysis.solve_power(
    effect_size = 0.451, 
    power = 0.95, 
    alpha = alpha_main, 
    alternative = 'larger'
)
sample_size = np.round(sample_size).astype(int) 
print('Sample size:', sample_size)

Sample size: 80


/home/johnv/anaconda3/envs/neuroforecasting/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:6832: RuntimeWarning: divide by zero encountered in _nct_sf
  return np.clip(_boost._nct_sf(x, df, nc), 0, 1)


In [4]:
def simulate(p_aware, n_subjects, seed = None):
    '''
    Simulates data that is null for the masked conditions, and has a nonzero
    effect for the unmasked conditions.
    
    All the DataFrame junk going on inside DataSimulator makes this way slower 
    but we get to use the same functions we'll use to analyze the real data this way.
    '''
    # simulate data
    sim = DataSimulator( # use simulation params computed from n = 190 dataset
        n_trials = 40, # reported at https://doi.org/10.31234/osf.io/4z2rj
        effect_size_ms = 28.,
        sd_pop = 76.,
        sd_sub = 92.
    )
    rng = np.random.default_rng(seed)
    dfs = [sim.simulate_subject(p_aware, rng) for sub in range(n_subjects)]
    means = [calc_overest_means(df) for df in dfs]
    df = pd.DataFrame(means)
    # compute paired differences
    delta_masked = df['masked operant'] - df['masked baseline']
    delta_unmasked = df['unmasked operant'] - df['unmasked baseline']
    delta2 = delta_masked - delta_unmasked
    # test for main effect of interest
    res = ttest_1samp(delta_masked, popmean = 0, alternative = 'greater')
    masked = res.pvalue < alpha_main
    # test for interaction effect
    res = ttest_1samp(delta2, popmean = 0, alternative = 'less')
    interaction = res.pvalue < alpha_inter # use nominal alpha, not adjusted
    return masked, interaction

In [5]:
## find combined false positive rate by simulation
pfunc = delayed(simulate)
parallel = Parallel(n_jobs = -1, verbose = 1)
output = parallel(pfunc(p_aware, sample_size, i) for i in range(n_simulations))
output = np.array(output)
fpr = output[:, 0].mean()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  38 tasks      | elapsed:   12.4s
[Parallel(n_jobs=-1)]: Done 188 tasks      | elapsed:   53.6s
[Parallel(n_jobs=-1)]: Done 438 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 788 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 1238 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-1)]: Done 1788 tasks      | elapsed:  8.5min
[Parallel(n_jobs=-1)]: Done 2438 tasks      | elapsed: 11.6min
[Parallel(n_jobs=-1)]: Done 3188 tasks      | elapsed: 15.2min
[Parallel(n_jobs=-1)]: Done 4038 tasks      | elapsed: 19.3min
[Parallel(n_jobs=-1)]: Done 4988 tasks      | elapsed: 23.9min
[Parallel(n_jobs=-1)]: Done 6038 tasks      | elapsed: 29.0min
[Parallel(n_jobs=-1)]: Done 7188 tasks      | elapsed: 34.6min
[Parallel(n_jobs=-1)]: Done 8438 tasks      | elapsed: 40.0min
[Parallel(n_jobs=-1)]: Done 9788 tasks      | elapsed: 45.8min
[Parallel(n_jobs=-1)]: Done 10000 out of 10000

In [6]:
print('The false positive is rate = %.02f%% at p_aware = %.02f...'%(100*fpr, p_aware))
print('Approximately %.02f%% due to statistical error,'%(100*alpha_main))
print('plus %.02f%% due to residual awareness.'%(100*(fpr - alpha_main)))

accuracy = .5*(1 - p_aware) + 1.*p_aware
print('\nTo be sure p_aware is less than %.02f,' \
    ' then accuracy must be less than %.02f%%.'%(p_aware, 100*accuracy))

The false positive is rate = 4.27% at p_aware = 0.05...
Approximately 1.00% due to statistical error,
plus 3.27% due to residual awareness.

To be sure p_aware is less than 0.05, then accuracy must be less than 52.50%.


In [7]:
power_interaction = output[:, 1].mean()
print('Power for detecting the interaction effect is %.02f.'%power_interaction)

Power for detecting the interaction effect is 0.91.
